In [1]:
import pandas as pd
import pyodbc
import useful_functions as use

In [2]:
#Read files
ppd = pd.read_csv('../../Data/PPD/ppd_analysis_file.csv', low_memory=False)
sample_valid = pd.read_excel('../../Data/Humach/2022-04-25_Validation_Sample.xlsx')
orginal_valid = len(sample_valid)
sample_standard = pd.read_excel('../../Data/Humach/2022-04-25_Masterfile_Random_Sample.xlsx')
orginal_standard = len(sample_standard)

In [3]:
#Remove missing records from samples
removed = len(sample_valid[~sample_valid.ENTITY_ID.isin(ppd.ENTITY_ID)])
print(f'{removed} entity ids in validation sample no longer in ppd')
sample_valid = sample_valid[sample_valid.ENTITY_ID.isin(ppd.ENTITY_ID)]
removed = len(sample_standard[~sample_standard.ENTITY_ID.isin(ppd.ENTITY_ID)])
print(f'{removed} entity ids in standard sample no longer in ppd')
sample_standard = sample_standard[sample_standard.ENTITY_ID.isin(ppd.ENTITY_ID)]

1 entity ids in validation sample no longer in ppd
12 entity ids in standard sample no longer in ppd


In [4]:
#3. get the existing no-contact data to filter out entities with phone category restriction. 
username = 'vigrose'
password_edw = 'Hufflepuff~10946'
password_aims = 'DQA8K@T7pXEqEt'
s = "DSN=aims_prod; UID={}; PWD={}".format(username, password_aims)
informix = pyodbc.connect(s)

sql_no_contacts = \
"""
    SELECT * FROM entity_cat_ct WHERE 
    end_dt is null AND category_code IN ("NO_CONTACT", "NO-EMAIL", "NO-RELEASE", "NO-CONTACT")
"""

no_contact_data = pd.read_sql(con=informix, sql=sql_no_contacts)

In [5]:
#Remove new no contacts from samples
removed = len(sample_valid[sample_valid.ENTITY_ID.isin(no_contact_data.entity_id)])
print(f'{removed} entity ids in validation sample are now no contact')
sample_valid = sample_valid[~sample_valid.ENTITY_ID.isin(no_contact_data.entity_id)]
removed = len(sample_standard[sample_standard.ENTITY_ID.isin(no_contact_data.entity_id)])
print(f'{removed} entity ids in standard sample are now no contact')
sample_standard = sample_standard[~sample_standard.ENTITY_ID.isin(no_contact_data.entity_id)]

0 entity ids in validation sample are now no contact
6 entity ids in standard sample are now no contact


In [6]:
#2. for each entity_id in sample, check corresponding phone number and address (via comm_id or comparing raw phone number and address)
sample_standard_2 = pd.merge(sample_standard, ppd[['ENTITY_ID', 'POLO_COMM_ID','TELEPHONE_NUMBER']], on=['ENTITY_ID', 'POLO_COMM_ID','TELEPHONE_NUMBER'])
sample_standard_updated = sample_standard[~sample_standard.ENTITY_ID.isin(sample_standard_2.ENTITY_ID)]
print(f'{len(sample_standard_updated)} addresses or phones on standard sample have been updated since sample was created')
sample_valid_2 = pd.merge(sample_valid, ppd[['ENTITY_ID', 'POLO_COMM_ID','TELEPHONE_NUMBER']], on=['ENTITY_ID', 'POLO_COMM_ID','TELEPHONE_NUMBER'])
sample_valid_updated = sample_valid[~sample_valid.ENTITY_ID.isin(sample_valid_2.ENTITY_ID)]
print(f'{len(sample_valid_updated)} addresses or phones on valid sample have been updated since sample was created')

322 addresses or phones on standard sample have been updated since sample was created
31 addresses or phones on valid sample have been updated since sample was created


In [8]:
print(f'Standard Sample: {len(sample_standard_2)} rows')
print(f'Validation Sample: {len(sample_valid_2)} rows')

Standard Sample: 34660 rows
Validation Sample: 2468 rows


In [10]:
sample_standard_2.ME = use.fix_me(sample_standard_2.ME)
sample_standard_2.to_excel('../../Data/Humach/2022-04-22_Masterfile_Random_Sample.xlsx', index=False)
sample_valid_2.ME = use.fix_me(sample_valid_2.ME)
sample_valid_2.to_excel('../../Data/Humach/2022-04-22_Validation_Sample.xlsx', index=False)